## Variación espacio-temporal precipitación total

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

Directorio

In [1]:
import os

print('> Directorio actual: ', os.getcwd())  
os.chdir('../')
print('> Directorio actual: ', os.getcwd()) 

> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics/cerro_saroche
> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics


In [2]:
import pandas as pd
import pickle

from utils.MONGO import CONEXION
from utils.UTILS import *
from datetime import datetime

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
park = 'cerro_saroche'
DIR = f'./{park}/'
experimento = f'experiments/narx/ndvi/'

### Cargando datos precipitación ANN

In [5]:
pd_precipitacion = pd.read_pickle(f'./{park}/data/ann_precipitacion.pkl')[['park',
                                                                            'periodo',
                                                                            'year',
                                                                            'month',
                                                                            'id_point',
                                                                            'latitud',
                                                                            'longitud',
                                                                            'type',
                                                                            'prediction_ann',
                                                                            'ndvi_media']]

# Transformacion
ndvi_transformacion = MinMaxScaler() #LogMinimax.create( pd_sst.oni.to_numpy() )
ndvi_transformacion.fit(pd_precipitacion[['prediction_ann','ndvi_media']])

pd_precipitacion[['precipitation_ann_t','ndvi_t']] = ndvi_transformacion.transform( pd_precipitacion[['prediction_ann','ndvi_media']] )
pd_precipitacion.head()

,park,periodo,year,month,id_point,latitud,longitud,type,prediction_ann,ndvi_media,precipitation_ann_t,ndvi_t
0,cerro_saroche,1995-01-01,1995,1,1,10.31,-69.83,training,0.207416,NaN,0.008535,NaN
1,cerro_saroche,1995-02-01,1995,2,1,10.31,-69.83,training,0.262076,NaN,0.028418,NaN
2,cerro_saroche,1995-03-01,1995,3,1,10.31,-69.83,training,0.698307,NaN,0.187100,NaN
3,cerro_saroche,1995-04-01,1995,4,1,10.31,-69.83,training,1.442033,NaN,0.457635,NaN
4,cerro_saroche,1995-05-01,1995,5,1,10.31,-69.83,training,1.287615,NaN,0.401464,NaN


## Resultados

In [6]:
# Resultados
experiments = [DIR+experimento + x for x in os.listdir(DIR+experimento)]
pd_summary = pd.concat(list(map(
                                lambda expe: pd.concat([pd.read_csv(expe + '/' +x) for x in os.listdir( expe ) if x.find('summary')!=-1 ]),
                                experiments)))

pd_summary.head()

,epocas,prediction_order,auto_order,exog_order,exog_delay,activation,id_point,n_neurons,capas,training_mse,...,validation_mse,validation_rmse,validation_mae,validation_mape,validation_r,test_mse,test_rmse,test_mae,test_mape,test_r
0,26,12,60,12,1,sigmoid,1,[48],1,0.061649,...,0.086185,0.007428,0.072089,13.200974,0.323491,0.094593,0.008948,0.083480,15.686368,0.185048
0,20,12,60,12,1,sigmoid,10,[48],1,0.051805,...,0.093717,0.008783,0.083539,19.674958,0.213820,0.102270,0.010459,0.091850,22.082628,0.063759
0,7,12,60,12,1,sigmoid,12,[48],1,0.079986,...,0.152825,0.023355,0.140674,31.787725,-3.677288,0.149733,0.022420,0.137168,30.911069,-3.489959
0,27,12,60,12,1,sigmoid,13,[48],1,0.051062,...,0.138164,0.019089,0.124345,26.590067,-2.373762,0.142623,0.020341,0.128219,27.408865,-2.595047
0,30,12,60,12,1,sigmoid,14,[48],1,0.049617,...,0.112416,0.012637,0.092573,20.431775,-1.180642,0.118942,0.014147,0.098899,21.801272,-1.441181


# Best model

In [7]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [8]:
best_model = pd_summary[pd_summary.test_r == pd_summary.test_r.max()]
dir_model = DIR+experimento+'15'

# Best model
model = keras.models.load_model(dir_model+'/model.h5')

In [9]:
# Configuracion
with open(f'{dir_model}/model_confi.pkl', 'rb') as f:
    model_confi = pickle.load(f)

#pd_model_id = model_confi['']
exog_order = model_confi['exog_order']
auto_order = model_confi['auto_order']
exog_delay = model_confi['exog_delay']
prediction_order = model_confi['prediction_order']
exogena = 'precipitation_ann_t'
y_output = 'ndvi_t'

In [10]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [11]:
list_ids = [int(x) for x in os.listdir(f'./{park}/experiments/narx/ndvi')]
list_ids

[1, 10, 12, 13, 14, 15, 2, 6, 7, 8, 9]

In [12]:
list_data = []

for id_point in list_ids:

    pd_model_id = pd_precipitacion[pd_precipitacion.id_point==id_point]
    pd_model_id.index = pd.to_datetime(pd_model_id.periodo)
    pd_model_id = pd_model_id[[y_output,exogena]].dropna().sort_index()

    # Datos
    x_data, y_data = split_data(pd_model_id,exog_order,auto_order,exog_delay,prediction_order,exogena,y_output)

    trainPredict = model.predict(x_data, verbose=0).reshape(-1)

    # Data de test
    trainind_pd = pd.DataFrame(trainPredict,
                                index = pd_model_id[-x_data.shape[0]:].index,
                                columns=['prediction']
                                )

    trainind_pd[y_output] = y_data.reshape(-1)
    trainind_pd['type'] = 'training'
    trainind_pd['precipitation_ann_t'] = np.nan

    trainind_pd['id_point'] = id_point

    trainind_pd[['prediction_ann','ndvi_prediction']] = ndvi_transformacion.inverse_transform(trainind_pd[['precipitation_ann_t','prediction']])
    trainind_pd[['prediction_ann','ndvi_media']] = ndvi_transformacion.inverse_transform(trainind_pd[['precipitation_ann_t',y_output]])

    trainind_pd = trainind_pd.reset_index(drop=False)[['id_point', 'periodo','type','ndvi_prediction','ndvi_media']]


    # Pronostico
    data_predict = pd_model_id[[y_output,exogena]]

    data_exogena = pd_precipitacion[(pd_precipitacion.periodo > data_predict.index.max()) & (pd_precipitacion.id_point==id_point)][[exogena,'periodo']]
    data_exogena.index = pd.to_datetime(data_exogena.periodo)
    data_exogena[y_output] = np.nan
    data_exogena = data_exogena.sort_index()[[exogena,y_output]]

    pd_prediction = predict_one_stap_narx(model,data_predict,data_exogena,exog_order,auto_order,exog_delay,prediction_order, exogena, y_output)
    pd_prediction = pd_prediction.rename(columns={y_output:'prediction'})
    pd_prediction['type'] = 'prediction'
    pd_prediction['id_point'] = id_point


    pd_prediction[['prediction_ann','ndvi_prediction']] = ndvi_transformacion.inverse_transform(pd_prediction[['precipitation_ann_t','prediction']])
    pd_prediction['ndvi_media'] = np.nan

    pd_prediction = pd_prediction.reset_index(drop=False)[['id_point', 'periodo','type','ndvi_prediction','ndvi_media']]


    # Uniendo informacion
    pd_summary = pd.concat([trainind_pd[list(pd_prediction)], 
                            pd_prediction[list(pd_prediction)]
                            ])

    list_data.append(pd_summary)

In [13]:
pd_analisis_ndvi = pd.concat(list_data)
pd_analisis_ndvi.head()

,id_point,periodo,type,ndvi_prediction,ndvi_media
0,1,2017-01-01,training,0.293772,0.573895
1,1,2017-02-01,training,0.282262,0.459801
2,1,2017-03-01,training,0.282016,0.446477
3,1,2017-04-01,training,0.310414,0.437340
4,1,2017-05-01,training,0.304862,0.480627


In [14]:
pd_periodo_null = pd_analisis_ndvi.query("id_point==1").copy()
pd_periodo_null['ndvi_prediction'] = np.nan 
pd_periodo_null['ndvi_media'] = np.nan 

def ndvi_model_trace(data,row, col, pd_periodo_null):
    """
    Funcion para la graficacion de la prediccion del ndvi
    """
    if data.shape[0]==0:
        data_figure_ajuste = pd_periodo_null[pd_periodo_null.type=='training'].sort_values('periodo')
        data_figure_pronostico = pd_periodo_null[pd_periodo_null.type=='prediction'].sort_values('periodo')
    else:
        data_figure_ajuste = data[data.type=='training'].sort_values('periodo')
        data_figure_pronostico = data[data.type=='prediction'].sort_values('periodo')



    if data.id_point.unique()==[1]:
        showlegend=True
    else:
        showlegend=False

    marker_line_width=1.5
    marker_size=1
    line_width=2


    fig.add_trace(go.Scatter(x=data_figure_ajuste['periodo'], y=data_figure_ajuste['ndvi_prediction'],
                             mode='lines+markers',name='Pronóstico entrenamiento',
                             marker_symbol='hexagram',
                             marker_line_width=marker_line_width,
                             marker_size=marker_size,
                             marker_line_color='#00D470',
                             marker_color='#00D470',
                             line=dict(color='#00D470', width=line_width),
                             legendgroup='group1',
                             showlegend=showlegend
                            ),     
            row=row, 
            col=col)

    fig.add_trace(go.Scatter(x=data_figure_ajuste['periodo'], y=data_figure_ajuste['ndvi_media'],
                             mode='lines+markers',name='NDVI entrenamiento',
                             marker_symbol='x-thin',
                             marker_line_width=marker_line_width,
                             marker_size=marker_size,
                             marker_line_color='#004A27',
                             marker_color='#004A27',
                             line=dict(color='#004A27', width=line_width),
                             legendgroup='group2',
                             showlegend=showlegend
                            ),     
            row=row, 
            col=col)

    fig.add_trace(go.Scatter(x=data_figure_pronostico['periodo'], y=data_figure_pronostico['ndvi_prediction'],
                             text=data_figure_pronostico['ndvi_prediction'].apply(lambda x: str(round(x,2)) ),
                             textposition="top right",
                             marker_symbol='star',
                             marker_line_width=marker_line_width,
                             marker_size=marker_size,
                             marker_line_color='#49FF00',
                             marker_color='#49FF00',
                             mode='lines+markers',name='Pronóstico NDVI',
                             line=dict(color='#49FF00', width=line_width,dash='dot'),
                            legendgroup='group5',
                            showlegend=showlegend
                            ),
            row=row, 
            col=col)

    fig.add_vline(x=data_figure_ajuste['periodo'].max(), line_width=2, line_dash="dash", line_color="#580606")
    
    fig.update_xaxes(tickformat="%Y/%m",
                     showline=True, 
                     linewidth=1, 
                     linecolor='black',
                     gridcolor='#E4E4E4',
                     mirror=True,
                     ticks="outside", 
                     tickwidth=2, 
                     tickcolor='#5C2B05',
                     ticklen=10,
                    range=[data.periodo.min(),
                           data.periodo.max()],
                    title_text='Mes',
                    title_font=dict(size=12)
                    )
    fig.update_yaxes(showline=True,
                     linewidth=1,
                     linecolor='black', 
                     gridcolor='#E4E4E4',
                     mirror=True,
                     ticks="outside", 
                     tickwidth=2, 
                     tickcolor='#5C2B05',
                     ticklen=10,
                     range=[0,1],
                     title_text='NDVI',
                     title_font=dict(size=12))

    return fig

In [15]:
pd_analisis_ndvi

,id_point,periodo,type,ndvi_prediction,ndvi_media
0,1,2017-01-01,training,0.293772,0.573895
1,1,2017-02-01,training,0.282262,0.459801
2,1,2017-03-01,training,0.282016,0.446477
3,1,2017-04-01,training,0.310414,0.437340
4,1,2017-05-01,training,0.304862,0.480627
...,...,...,...,...,...
7,9,2023-01-01,prediction,0.256436,NaN
8,9,2023-02-01,prediction,0.236565,NaN
9,9,2023-03-01,prediction,0.249577,NaN
10,9,2023-04-01,prediction,0.264667,NaN


In [16]:
rows=5
cols=3

list_id = list(range(1,16)) 

column_titles = list(map(lambda x: 'Punto id:' + str(int(x)), list_id))

fig = make_subplots(rows=rows,
                    cols=cols,
                    subplot_titles=column_titles,
                    horizontal_spacing=0.1,
                    vertical_spacing=0.13)


row_order = np.array(list_id).reshape(rows,cols).tolist()


for row in range(len(row_order)):

    list_row = row_order[row]
    for col in range(len(list_row)):
        

        data_fig = pd_analisis_ndvi\
                        .query(f"id_point=={row_order[row][col]}")\
                        .sort_values('periodo',ascending=True)

        ndvi_model_trace(data_fig, row=row+1, col=col+1,pd_periodo_null=pd_periodo_null)


#------------
fig['layout']['title']['y']=0.98
fig['layout']['margin']['t']=100

#------------
annotations_title = fig['layout']['annotations']

y_loc=0.8
yshift=4
annotations = []

for id_ in  list_id:
    
    xref='x{id_}'.format(id_=str(int(id_)))
    yref='y{id_}'.format(id_=str(int(id_)))
    
    titulo_annotations =  list(filter(lambda x: x.text=='Punto id:{id_}'.format(id_=int(id_)),
                                  annotations_title))[0]

    annotations= annotations+[
                dict(font=dict(size=16),
                    showarrow=titulo_annotations['showarrow'],
                    text=titulo_annotations['text'],
                    x=titulo_annotations['x'],
                    xanchor=titulo_annotations['xanchor'],
                    xref='paper',#xref,
                    y=titulo_annotations['y'],
                    yanchor=titulo_annotations['yanchor'],
                    yref='paper',#yref
                    ),
                dict(
                    x=pd_analisis_ndvi.query("type=='training'").periodo.max() + pd.DateOffset(months=6),
                    y=y_loc, # annotation point
                    xref=xref, 
                    yref=yref, 
                    text='Pronóstico',
                    showarrow=False,
                    yshift=yshift,
                    font=dict(size=9)
                  )
                    ]

fig['layout']['annotations'] = annotations

        
#------------       
fig.update_xaxes(tickformat="%Y/%m",showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)   

#------------
data_figure_pronostico = pd_analisis_ndvi.query("type=='prediction'").copy()
fig.update_layout( height=800,
                   width=1500,
                   #showlegend = True,
                   font = dict(size = 12),
                   template='plotly_white',
                   title_text="""NDVI Cerro Saroche
                        <br><sup>Pronóstico para el periodo {date_init} al {date_fin}</sup>
                        """.format(date_init=str(data_figure_pronostico['periodo'].min().strftime('%Y/%m')),
                                   date_fin=str(data_figure_pronostico['periodo'].max().strftime('%Y/%m'))),
                  legend_title_text='Serie',
                  legend_title = dict( font = dict(size = 14)),
                   uniformtext_minsize=8,
                   uniformtext_mode='hide',
                   legend = dict(font_size=14)
                 )

fig.write_image(f'./{park}/figures/narx_ndvi.png')

fig.show()

In [17]:
pd_data_test = pd_analisis_ndvi.query("type=='training'")

list_metricas = []
for id in pd_data_test.sort_values('id_point').id_point.unique():
    pd_data = pd_data_test.query(f"id_point=={id}").sort_values('id_point')
    pd_metricas = pd.DataFrame(metrics(observado=pd_data.ndvi_media,
                                       prediccion=pd_data.ndvi_prediction)
                                ,index=[0])

    pd_metricas['id_point'] = id

    list_metricas.append(pd_metricas)

pd_metricas_ndvi = pd.concat(list_metricas) 
pd_metricas_ndvi

,mape,mae,mse,rmse,r2,id_point
0,36.510656,0.197584,0.220900,0.048797,-2.992823,1
0,35.513374,0.176551,0.200222,0.040089,-2.455168,2
0,35.790032,0.176261,0.196383,0.038566,-2.842326,6
0,35.255884,0.139940,0.156254,0.024415,-2.285785,7
0,35.521869,0.141081,0.158582,0.025148,-2.234149,8
0,32.675847,0.128346,0.144587,0.020905,-2.499825,9
0,31.944763,0.130356,0.146427,0.021441,-2.190742,10
0,34.718341,0.127193,0.142240,0.020232,-2.287698,12
0,36.156251,0.137851,0.151413,0.022926,-2.811800,13
0,31.503516,0.115952,0.132852,0.017650,-2.188390,14
